In [2]:
import numpy as np
#from scipy.signal import butter, filtfilt
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
#import seaborn as sns
#sns.set_context("notebook", font_scale=1.3,
#                rc={'lines.linewidth': 1.5, 'lines.markersize': 14, 'axes.titlesize': 'x-large'})
#matplotlib.rc('legend', numpoints=1, fontsize=14)
#import glob
import sys, os
sys.path.insert(1, r'./Functions')
# IPython widgets:
from IPython.display import display
import ipywidgets
from ipywidgets import FloatProgress, interactive
import seaborn as sns
from scipy.signal import detrend

In [3]:
#!pip install version_information
%load_ext version_information
%version_information numpy, scipy, pandas, matplotlib, seaborn

Software versions
Python 3.8.8 64bit [MSC v.1916 64 bit (AMD64)]
IPython 7.22.0
OS Windows 10 10.0.22000 SP0
numpy 1.20.1
scipy 1.6.2
pandas 1.2.4
matplotlib 3.3.4
seaborn 0.11.1
Fri Oct 07 14:32:58 2022 Hora oficial do Brasil

In [4]:
# GitHub URL:
#path2 = 'C:\Users\ferna\Downloads\IC- dados\Dados'
# local directory:
path2 = r'C:\\Users\\ferna\\Downloads\\IC- dados\\Dados'
fname = os.path.join(path2, 'PDSinfo.txt')
PDSinfo = pd.read_csv(fname, sep='\t', header=0, index_col=None, engine='c', encoding='utf-8')
print(fname)
print("Information of %s subjects loaded (%s rows, %s columns)."
      %(len(pd.unique(PDSinfo.Subject)), PDSinfo.shape[0], PDSinfo.shape[1]))

C:\\Users\\ferna\\Downloads\\IC- dados\\Dados\PDSinfo.txt
Information of 49 subjects loaded (588 rows, 29 columns).


In [5]:
a=PDSinfo[PDSinfo['Subject']==44].index.tolist()
e=PDSinfo[PDSinfo['Subject']==44].index.tolist()[2]

In [6]:

fs = 100 # Hz
COPX = np.zeros((0, 60*fs-30))
COPZ = np.zeros((0, 60*fs-30))

COPXreal = np.zeros((0, 60*fs-30))
COPYreal = np.zeros((0, 60*fs-30))
COPZreal = np.zeros((0, 60*fs-30))

d1x = np.zeros((0, 60*fs-30))
d2x = np.zeros((0, 60*fs-30))

d1z = np.zeros((0, 60*fs-30))
d2z = np.zeros((0, 60*fs-30))

matrizangx = np.zeros((0, 60*fs-30))
matrizangz = np.zeros((0, 60*fs-30))

alpha = np.zeros((0, 60*fs-30))

COGX = np.zeros((0, 60*fs-30))
COGY = np.zeros((0,60*fs-30))
COGZ = np.zeros((0, 60*fs-30)) 
 

subject = np.zeros((0,1))
j=0

for s in range (1,50):
    a = PDSinfo[PDSinfo['Subject']==s].index.tolist()
    for i in range(len(a)):
        fname_grf = os.path.join(path2, PDSinfo.Trial[a[i]] + 'grf' + '.txt') 
        fname_mkr = os.path.join(path2, PDSinfo.Trial[a[i]] + 'mkr' + '.txt')
        grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
        mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
        
        if not np.any(np.isnan(mkr['COG_X'].values[30:])):
            COGX = np.vstack((COGX, mkr['COG_X'].values[30:]))
            COGY = np.vstack((COGY, mkr['COG_Y'].values[30:]))
            COGZ = np.vstack((COGZ, mkr['COG_Z'].values[30:]))
    
            mass=PDSinfo['Mass']
            d = np.sqrt(COGX[s-1,:]**2+COGY[s-1,:]**2+COGZ[s-1,:]**2)
            angulox = np.arcsin(COGX[s-1,:]/d)
            anguloz= np.arcsin(COGZ[s-1,:]/d)
            sinx=np.sin(angulox)
            sinz=np.sin(anguloz)
            cosx=np.cos(angulox)
            cosz=np.cos(anguloz)
            tanx=np.tan(angulox)
            tanz=np.tan(anguloz)
            matrizangx = np.vstack((matrizangx,angulox))
            matrizangz = np.vstack((matrizangz,anguloz))


            Jb=(4/3)*(mass.values[s])*((d)**2)
            t=grf['Time']
            dt=t[1]-t[0]

            d1x = np.vstack((d1x, np.gradient(matrizangx[s-1,:],dt)))
            d1z = np.vstack((d1z, np.gradient(matrizangz[s-1,:],dt)))

            d2x = np.vstack((d2x, np.gradient(d1x[s-1,:],dt)))
            d2z = np.vstack((d2z, np.gradient(d1z[s-1,:],dt)))

            COPX = np.vstack((COPX, d*sinx + Jb/(mass.values[s]*9.8)*(sinx*d1x[s-1,:]**2-cosx*d2x[s-1,:])))
            COPZ = np.vstack((COPZ, d*sinz + Jb/(mass.values[s]*9.8)*(sinz*d1z[s-1,:]**2-cosz*d2z[s-1,:])))


            COPXreal = np.vstack((COPXreal, grf['COPNET_X'].values[30:]))
            COPZreal = np.vstack((COPZreal, grf['COPNET_Z'].values[30:]))
            
            subject = np.vstack((subject, [s]))
    

    
print(COGX.shape)
print(COPX.shape)

(574, 5970)
(574, 5970)


In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
fs = 100 # Hz
COPX = np.zeros((49, 60*fs-30))
COPZ = np.zeros((49, 60*fs-30))

COPXreal = np.zeros((49, 60*fs-30))
COPYreal = np.zeros((49, 60*fs-30))
COPZreal = np.zeros((49, 60*fs-30))

d1x = np.zeros((49, 60*fs-30))
d2x = np.zeros((49, 60*fs-30))

d1z = np.zeros((49, 60*fs-30))
d2z = np.zeros((49, 60*fs-30))

matrizangx = np.zeros((49, 60*fs-30))
matrizangz = np.zeros((49, 60*fs-30))

alpha = np.zeros((49, 60*fs-30))


ASISX = np.zeros((49, 60*fs-30))
ASISY = np.zeros((49,60*fs-30))
ASISZ = np.zeros((49, 60*fs-30))

subject = np.zeros((0,1))
j=0

for s in range (1,50):
    a = PDSinfo[PDSinfo['Subject']==s].index.tolist()[2]
    
    fname_grf = os.path.join(path2, PDSinfo.Trial[a] + 'grf' + '.txt') 
    fname_mkr = os.path.join(path2, PDSinfo.Trial[a] + 'mkr' + '.txt')
    grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
    mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
    colunas = mkr.columns
    #print(list(colunas))
    ASISX[s-1,:]=mkr['R.ASIS_X'].values[30:]
    ASISY[s-1,:]=mkr['R.ASIS_Y'].values[30:]
    ASISZ[s-1,:]=mkr['R.ASIS_Z'].values[30:]
    #print(COGX)
    mass=PDSinfo['Mass']
    d = np.sqrt(ASISX[s-1,:]**2+ASISY[s-1,:]**2+ASISZ[s-1,:]**2)
    angulox = np.arcsin(ASISX[s-1,:]/d)
    anguloz= np.arcsin(ASISZ[s-1,:]/d)
    sinx=np.sin(angulox)
    sinz=np.sin(anguloz)
    cosx=np.cos(angulox)
    cosz=np.cos(anguloz)
    tanx=np.tan(angulox)
    tanz=np.tan(anguloz)
    matrizangx[s-1,:] = angulox
    matrizangz[s-1,:] = anguloz
    
    
    
    Jb=(4/3)*(mass.values[s])*((d)**2)
    t=grf['Time']
    dt=t[1]-t[0]

    d1x[s-1,:]=np.gradient(matrizangx[s-1,:],dt)
    d1z[s-1,:]=np.gradient(matrizangz[s-1,:],dt)

    d2x[s-1,:]=np.gradient(d1x[s-1,:],dt)
    d2z[s-1,:]=np.gradient(d1z[s-1,:],dt)
    
    COPX[s-1,:] = d*sinx + Jb/(mass.values[s]*9.8)*(sinx*d1x[s-1,:]**2-cosx*d2x[s-1,:])
    COPZ[s-1,:] = d*sinz + Jb/(mass.values[s]*9.8)*(sinz*d1z[s-1,:]**2-cosz*d2z[s-1,:])
    
   
    
    
    COPXreal[s-1,:]=grf['COPNET_X'].values[30:]
    COPZreal[s-1,:]=grf['COPNET_Z'].values[30:]
    
    subject = np.vstack((subject, [s]))
    
    constant = np.ones_like(COPX[s-1,:])

    
    x= np.vstack((COPX[[s-1],:],constant))
    y= COPXreal[s-1,:]

    model=LinearRegression(fit_intercept=False)
    model.fit(x.T,y)
    
    
    #split=train_test_split(COGX,COPXreal,test_size=9)
   # COGXtrain = split[0]
    #COPXtrain = split[1]
    #COGXtest = split[2]
   # COPXtest = split[3]
    
#print(COGXtrain)
#for i in COGXtrain:
 #   i=0
#print(COGXtrain[i])
    
    
    
    
    #print(model.coef_) 
    
    #def reg_m(y, x):
        #ones = np.ones(len(x[0]))
        #X = sm.add_constant(np.column_stack((x[0], ones)))
        #for oi in x[1:]:
            #X = sm.add_constant(np.column_stack((oi, X)))
            #results = sm.OLS(y, x).fit()
            #return results
    #print(model.coef_)
    #print(reg_m(y, x.T).summary())
 
    #print(model.coef_)
    #x= (np.array([COPX[s-1,:],d2x[s-1,:]]))
    #y= COPXreal[s-1,:]
    #model=LinearRegression()
    #model.fit(x.T,y)
    #print(model.coef_) 
    #Histogramas
    #plt.figure()
    #hist=plt.hist(model.coef_[0], bins=50)
    #plt.title('Histograms of alpha'.format(s))
    #plt.show()


In [8]:
indices = np.arange(1,50).reshape(-1,1)
np.random.shuffle(indices)
train = indices[:40]
test = indices[40:]



np.in1d(subject,train)
ASISXtrain = ASISX[np.in1d(subject,train),:]
ASISXtest = ASISX[np.in1d(subject,test),:]
COPXtrain = COPX[np.in1d(subject,train),:]
COPXtest = COPX[np.in1d(subject,test),:]
ASISYtrain = ASISY[np.in1d(subject,train),:]
ASISYtest = ASISY[np.in1d(subject,test),:]

ASISZtrain = ASISZ[np.in1d(subject,train),:]
ASISZtest = ASISZ[np.in1d(subject,test),:]
COPZtrain = COPZ[np.in1d(subject,train),:]
COPZtest = COPZ[np.in1d(subject,test),:]



In [25]:
from sklearn.model_selection import train_test_split
alpha= []
beta=[]
gama = []
#A = []
for i in range(0,len(ASISXtrain)):
    d = np.sqrt(ASISXtrain[i]**2+ASISYtrain[i]**2+ASISZtrain[i]**2)
    angulox = np.arcsin(ASISXtrain[i]/d)
    anguloz = np.arcsin(ASISZtrain[i]/d)
    sinx=np.sin(angulox)
    sinz=np.sin(anguloz)
    cosx=np.cos(angulox)
    cosz=np.cos(anguloz)
    tanx=np.tan(angulox)
    tanz=np.tan(anguloz)
    matrizangx[i,:] = angulox
    matrizangz[i,:] = anguloz
    
    #A = np.zeros_like(angulox)
    
    sennovo = sinx*A + ((1-A**2)**1/2)*cosx
    
    cosnovo = cosx*A - ((1-A**2)**1/2)*sinx
     
    Jb=(4/3)*(mass.values[s])*((d)**2)
    t=grf['Time']
    dt=t[1]-t[0]

    d1x[i,:]=np.gradient(matrizangx[i,:],dt)
    d1z[i,:]=np.gradient(matrizangz[i,:],dt)
    
    

    d2x[i,:]=np.gradient(d1x[i,:],dt)
    d2z[i,:]=np.gradient(d1z[i,:],dt)
    
    COPX[i,:] = d*sennovo + Jb/(mass.values[s]*9.8)*(sennovo*d1x[i,:]**2-cosnovo*d2x[i,:])
    COPZ[i,:] = d*sinz + Jb/(mass.values[s]*9.8)*(sinz*d1z[i,:]**2-cosz*d2z[i,:])
    
    constant = np.ones_like(COPX[i,:])

    
    
    
    x= np.vstack((COPX[i,:],d2x[i,:], constant))
    y= COPXtrain[i,:]
            
    

    model=LinearRegression(fit_intercept=False)
    model.fit(x.T,y)
    
    print(model.coef_)
    
    
    alpha.append(model.coef_[0])
    
    beta.append(model.coef_[1])
    
    gama.append(model.coef_[2])
   
alfamedia=np.mean(alpha)
betamedia=np.mean(beta)
gamamedia =np.mean(gama)
print(alfamedia)
print(betamedia)
print(gamamedia)

[-2.32952778 -0.08411025  1.43600237]
[-10.79777404   0.12572866   5.91613793]
[ 0.7301717  -0.15677169 -0.01266518]
[-15.77690355   0.18656521   8.30260576]
[-18.33282913   0.27622018   9.85187469]
[-9.441329    0.05041249  4.99941028]
[-9.93014806  0.06771638  4.72044927]
[-12.97980749   0.14728643   7.09076272]
[-10.20458797   0.0808707    5.03635936]
[-10.98029295   0.12597845   6.20828953]
[-17.86328715   0.21694514   7.88751186]
[-15.35186186   0.19786811   7.94157791]
[-13.61329017   0.13282883   6.51018269]
[-7.70922575e+00  6.42245883e-03  3.49259789e+00]
[-8.02074087  0.05083415  4.27571946]
[-15.37352048   0.14538289   7.71701782]
[ 1.96366681 -0.15608931 -0.51692434]
[-15.39605371   0.21403952   7.72048115]
[-7.46340772  0.02205155  4.35874483]
[ 2.51268504 -0.17744541 -0.86016486]
[-6.98117876e+00 -3.85951323e-03  3.56010555e+00]
[-12.31177233   0.142775     6.11334529]
[-5.88486372 -0.00980965  3.21932254]
[-7.85380572  0.05057671  4.03323706]
[ 2.00543665 -0.16846245 -0.

In [48]:
def centerofpressure ():
    from sklearn.model_selection import train_test_split
from scipy.optimize import minimize, rosen, rosen_der
alpha= []
beta=[]
gama = []
for i in range(0,len(ASISXtrain)):
    d = np.sqrt(ASISXtrain[i]**2+ASISYtrain[i]**2+ASISZtrain[i]**2)
    angulox = np.arcsin(ASISXtrain[i]/d)
    anguloz = np.arcsin(ASISZtrain[i]/d)
    sinx=np.sin(angulox)
    sinz=np.sin(anguloz)
    cosx=np.cos(angulox)
    cosz=np.cos(anguloz)
    tanx=np.tan(angulox)
    tanz=np.tan(anguloz)
    matrizangx[i,:] = angulox
    matrizangz[i,:] = anguloz
    
    sennovo = sinx*A + ((1-A**2)**1/2)*cosx
    
    cosnovo = cosx*A - ((1-A**2)**1/2)*sinx
     
    Jb=(4/3)*(mass.values[s])*((d)**2)
    t=grf['Time']
    dt=t[1]-t[0]

    d1x[i,:]=np.gradient(matrizangx[i,:],dt)
    d1z[i,:]=np.gradient(matrizangz[i,:],dt)
    
    

    d2x[i,:]=np.gradient(d1x[i,:],dt)
    d2z[i,:]=np.gradient(d1z[i,:],dt)
    
    COPX[i,:] = d*sennovo + Jb/(mass.values[s]*9.8)*(sennovo*d1x[i,:]**2-cosnovo*d2x[i,:])
    COPZ[i,:] = d*sinz + Jb/(mass.values[s]*9.8)*(sinz*d1z[i,:]**2-cosz*d2z[i,:])
    
    

In [44]:
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize, rosen, rosen_der
alpha= []
beta=[]
gama = []
for i in range(0,len(ASISXtrain)):
    d = np.sqrt(ASISXtrain[i]**2+ASISYtrain[i]**2+ASISZtrain[i]**2)
    angulox = np.arcsin(ASISXtrain[i]/d)
    anguloz = np.arcsin(ASISZtrain[i]/d)
    sinx=np.sin(angulox)
    sinz=np.sin(anguloz)
    cosx=np.cos(angulox)
    cosz=np.cos(anguloz)
    tanx=np.tan(angulox)
    tanz=np.tan(anguloz)
    matrizangx[i,:] = angulox
    matrizangz[i,:] = anguloz
    
    sennovo = sinx*A + ((1-A**2)**1/2)*cosx
    
    cosnovo = cosx*A - ((1-A**2)**1/2)*sinx
     
    Jb=(4/3)*(mass.values[s])*((d)**2)
    t=grf['Time']
    dt=t[1]-t[0]

    d1x[i,:]=np.gradient(matrizangx[i,:],dt)
    d1z[i,:]=np.gradient(matrizangz[i,:],dt)
    
    

    d2x[i,:]=np.gradient(d1x[i,:],dt)
    d2z[i,:]=np.gradient(d1z[i,:],dt)
    
    COPX[i,:] = d*sennovo + Jb/(mass.values[s]*9.8)*(sennovo*d1x[i,:]**2-cosnovo*d2x[i,:])
    COPZ[i,:] = d*sinz + Jb/(mass.values[s]*9.8)*(sinz*d1z[i,:]**2-cosz*d2z[i,:])
    
    constant = np.ones_like(COPX[i,:])
    
    x0 = [0,0,0,0]
    fun = d*sennovo + Jb/(mass.values[s]*9.8)*(sennovo*d1x[i,:]**2-cosnovo*d2x[i,:])
    res=minimize(fun,x0, method = 'BFGS', jac = rosen_der, options ={'gtol':1e-6, 'disp': True})

res.x


TypeError: 'numpy.ndarray' object is not callable

In [37]:
x0 = [1.4, 7, 8, 44]
fun = lambda x: (x[0]-1)**2 + (x[1] - 2.5)**2
from scipy.optimize import minimize, rosen, rosen_der
res = minimize(fun, (2,0), method ='BFGS', jac= rosen_der, options = {'gtol': 1e-6, 'disp': True })
res.x



         Current function value: 1.345773
         Iterations: 14
         Function evaluations: 73
         Gradient evaluations: 61


array([1.16250046, 1.35136312])

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
COPXES = np.zeros((len(ASISXtest), 60*fs-30))
COPXESorig =np.zeros((len(ASISXtest), 60*fs-30))
for u in range(0,len(ASISXtest)):
    d = np.sqrt(ASISXtest[u]**2+ASISYtest[u]**2+ASISZtest[u]**2)
    angulox = np.arcsin(ASISXtest[u]/d)
    anguloz = np.arcsin(ASISZtest[u]/d)
    sinx=np.sin(angulox)
    sinz=np.sin(anguloz)
    cosx=np.cos(angulox)
    cosz=np.cos(anguloz)
    tanx=np.tan(angulox)
    tanz=np.tan(anguloz)
    matrizangx[u,:] = angulox
    matrizangz[u,:] = anguloz
    
    Jb=(4/3)*(mass.values[s])*((d)**2)
    t=grf['Time']
    dt=t[1]-t[0]

    d1x[u,:]=np.gradient(matrizangx[u,:],dt)
    d1z[u,:]=np.gradient(matrizangz[u,:],dt)
    
    

    d2x[u,:]=np.gradient(d1x[u,:],dt)
    d2z[u,:]=np.gradient(d1z[u,:],dt)
    
    
    COPX[u,:] = d*sinx + Jb/(mass.values[s]*9.8)*(sinx*d1x[u,:]**2-cosx*d2x[u,:])
    
    
    #COPXES[u,:] = alfamedia* COPX[u,:] + betamedia*(d2x[u,:])
    
    COPXES[u,:] = alfamedia*(d*sinx + ((Jb/(mass.values[s]*9.8))*sinx*d1x[u,:]**2)) + d2x[u,:]*(-alfamedia*cosx*Jb/(mass.values[s]*9.8)+betamedia)+ gamamedia
    COPXESorig[u,:]= COPX[u,:]


time = np.arange(0,6000)/100
for i in range(0,len(ASISXtest)):
    plt.figure()
    plt.plot(time[:5970],detrend(COPXES[i,:]),color='purple',label='COP predicted')
    plt.plot(time[:5970],detrend(COPXreal[i]),color='blue', label='COP real')
    plt.title('COP real X COP calculated for test {:.1f}'.format(i))
    plt.xlabel('Time')
    plt.legend()
    plt.show()


Error from COPXESorig 

In [ ]:
from sklearn.metrics import mean_squared_error
ytrue = COPXtest
ypredicted = COPXESorig
mean_squared_error(ytrue, ypredicted)

In [ ]:
erro= (COPXtest - COPXESorig)**2
print(erro.mean())

Error from COPXES

In [ ]:
from sklearn.metrics import mean_squared_error
ytrue = COPXtest
ypredicted = COPXES
mean_squared_error(ytrue, ypredicted)

In [ ]:
erro= (COPXtest - COPXES)**2
print(erro.mean())